<a href="https://colab.research.google.com/github/COMP4702-UQ/Pracs-notebook/blob/main/PracW13-Tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# COMP4702/7703 - Machine Learning
# Prac W13: Generative Models

# Autoencoders

The code in this exercise is based on the "Introduction to autoencoders" tutorial available at: https://www.tensorflow.org/tutorials/generative/autoencoder

We start by building an autoencoder and training it on the fashion MNIST data.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, losses
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Model
#---
(x_train, _), (x_test, _) = fashion_mnist.load_data()

x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.

print (x_train.shape)
print (x_test.shape)
#---
class Autoencoder(Model):
  def __init__(self, latent_dim, shape):
    super(Autoencoder, self).__init__()
    self.latent_dim = latent_dim
    self.shape = shape
    self.encoder = tf.keras.Sequential([
      layers.Flatten(),
      layers.Dense(latent_dim, activation='relu'),
    ])
    self.decoder = tf.keras.Sequential([
      layers.Dense(tf.math.reduce_prod(shape).numpy(), activation='sigmoid'),
      layers.Reshape(shape)
    ])

  def call(self, x):
    encoded = self.encoder(x)
    decoded = self.decoder(encoded)
    return decoded


shape = x_test.shape[1:]
latent_dim = 64
autoencoder = Autoencoder(latent_dim, shape)
#---
autoencoder.compile(optimizer='adam', loss=losses.MeanSquaredError())

In [ ]:
#Training
#---
autoencoder.fit(x_train, x_train,
                epochs=10,
                shuffle=True,
                validation_data=(x_test, x_test))
#---
encoded_imgs = autoencoder.encoder(x_test).numpy()
decoded_imgs = autoencoder.decoder(encoded_imgs).numpy()
#---
n = 10
plt.figure(figsize=(20, 4))
for i in range(n):
  # display original
  ax = plt.subplot(2, n, i + 1)
  plt.imshow(x_test[i])
  plt.title("original")
  plt.gray()
  ax.get_xaxis().set_visible(False)
  ax.get_yaxis().set_visible(False)

  # display reconstruction
  ax = plt.subplot(2, n, i + 1 + n)
  plt.imshow(decoded_imgs[i])
  plt.title("reconstructed")
  plt.gray()
  ax.get_xaxis().set_visible(False)
  ax.get_yaxis().set_visible(False)
plt.show()

I think this example uses a weird autoencoder - it has a single layer of 64 units, with ReLU activations. Note that if we use linear activation functions in the hidden layer, the network can perform the equivalent of PCA.

**Q1**. Modify the network to use linear activation. Experiment to see if the performance is better/worse. Try training for longer and/or using stochastic gradient descent (sgd) as the optimizer.

**Q2**. Experiment with a smaller/larger number of neurons in the hidden layer. Note that (apart from numerical issues) it should be possible to achieve 0 error if we have 784 hidden units (why?).

**Q3**. Modify the network to be a proper nonlinear autoencoder. That is, you should have at least 3 hidden layers. It is not nescessary, but typically people make the structure symmetric - e.g. for this data: 784-hx-hy-hx-784. Experiment with training your network. It should be able to get a lower error, but it might take more training epochs.

**Q4**. To use this network as a generative model, we can take the encoder part of the (trained) network, create "fake" input patterns for it, and see what output/image is reconstructed. Do this for one of your models from Q1-3 above. Experiment with different methods of creating fake input patterns.

# Generative Adversarial Networks

**Q5**. Read the first few pages of Chapter 15 of The Understanding Deep Learning book (https://udlbook.github.io/udlbook/). Then, work through Notebook 15.1 from that book.

**Q6**. Read/work through this guide to building a GAN for the MNIST dataset:

https://machinelearningmastery.com/how-to-develop-a-generative-adversarial-network-for-an-mnist-handwritten-digits-from-scratch-in-keras/
